# GLM Build 

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import sklearn

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from sklearn.preprocessing import OrdinalEncoder


In [ ]:
Saga_NB_df = pd.read_csv(r"R:\Pricing & Actuarial\Pricing\Personal Lines Pricing - Motor\Technical\8. Optimisation\6. CDL\1. Jan22\2. Modelling\Python\Cancellation_NB_Saga_Nov19-Nov21_train.csv")

In [ ]:
Saga_NB_df.shape

In [ ]:
Saga_NB_df.info()

In [ ]:
Saga_NB_df.head()

In [ ]:
columns = Saga_NB_df.columns
new_column_headers = []
for val in columns:
    altered_header = val.replace(" ", "_")
    #print(val)
    #print(altered_header)
    new_column_headers.append(altered_header)

Saga_NB_df.columns = new_column_headers
print(Saga_NB_df.columns)

In [ ]:
#These are factors that either aren't GIPP compliant, allowed to be used in this stage of modelling  (may be added in overlays etc) or are part of the setup of the dataset, not important for modelling.
Saga_NB_dropped_unnecessary_df = Saga_NB_df.drop(columns=['Weight', 'set', 'Add_Driver_Sex', 'Main_Driver_Sex', 'Main_Driver_Age_Months'])


## Banded variables-use Ordinal Encoder

In [ ]:
annual_mileage_strings = []
for i in range(0,31,1):
    annual_mileage_strings.append(str(i))
print(annual_mileage_strings)


In [ ]:
Saga_NB_dropped_unnecessary_df

In [ ]:

Saga_NB_transformed_df = Saga_NB_dropped_unnecessary_df[[
                            'AgePassedTest', 'Cover', 'Duration', 'Garaged', 'NCDAllowed',
                            'NCDDifference', 'RegistrationYear', 'VehicleAgeatPurchase', 
                            'YearsOwned', 'Class_of_Use', 'Annual_Mileage', 'Age_Difference',
                            'Add_Driving_Experience', 'Add_Other_Vehs_Owned', 'Add_Driver_Age',
                            'Add_Access_to_Other_Vehs', 'Access_to_Other_Vehs', 'PDR_Code',
                            'Other_Vehs_Owned', 'NCD_Protected', 'NCD_Earned', 'Most_Severe_Conviction',
                            'Main_UK_Residency', 'Main_Occupation', 'Main_Marital_Status', 
                            'Main_Licence_Type', 'Main_Full_Time_Employed', 'Main_Employment_Type',
                            'Main_Driving_Experience_Months', 'Main_Employer_Business', 'Main_Driving_Experience',
                            'Latest_Conviction', 'Latest_Fault_Accident_Claim','Latest_Non_Fault_Accident_Claim',
                            'Latest_Windscreen_Claim','Main_Driver_Age', 'Transaction_Type', 'Tot_Windscreen_Claims', 
                            'Tot_Vandalism_Claims', 'Tot_Theft_Claims', 'Tot_Non_Fault_Accident_Claims', 
                            'Tot_Fire_Claims', 'Tot_Fault_Accident_Claims', 'Vehicle_Age', 'Vehicle_Keeper', 
                            'Vehicle_Owner', 'Vehicle_Value', 'Saga_Years_with_Broker', 'SagaCais', 'SagaFactor', 'SagaQuoteLag']]
ordinal_encoder = OrdinalEncoder(categories=[[-1, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87],
                                             ['Comp', 'TPFT'],
                                             [0,1,2,3,4,5,6],
                                             ['Car Park', 'Carport', 'Garaged', 'Locked Building', 'Locked Compound', 'On Drive', 'Private Property', 'Public Road', 'Third Party Premises', 'Unlocked Building', 'Unlocked Compund', 'Business Address', 'N/A'],
                                             [0,1,2,3,4,5,6,7,8,9], #Missing? Can't mix int and strings... what to do?
                                             [0,1,2,3,4,5,6,7,8,9],
                                             [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021],
                                             [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
                                             [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
                                             ['SDP', 'SDP & C', 'Class 1', 'Class 2', 'Class 3', 'Class 1 (Policyholder Only)', 'Class 1 (Spouse Only)'],
                                             ['1 - 1000', '1001 - 2000', '2001 - 3000', '3001 - 4000', '4001 - 5000', '5001 - 6000', '6001 - 7000', '7001 - 8000', '8001 - 9000', '9001 - 10000', '10001 - 11000', '11001 - 12000', '12001 - 13000', '13001 - 14000', '14001 - 15000', '15001 - 16000', '16001 - 17000', '17001 - 18000', '18001 - 19000', '19001 - 20000', '20001 - 21000', '21001 - 22000', '22001 - 23000', '23001 - 24000', '24001 - 25000', '25001 - 26000', '26001 - 27000', '27001 - 28000', '28001 - 29000', '29001 - 30000', 'Over 30000'],
                                             ['IOD', '-15+', '-10 to -14', '-5 to -9', '0 to -4',  '1 to 5','6+'],
                                             ['IOD', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
                                             ['IOD', '0', '1', '2', '3', '4'],
                                             ['IOD', '17 to 24', '25 to 29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65+'],
                                             ['IOD', '0', '1', '2', '3', '4'],
                                             [0,1,2,3,4,5],
                                             ['Any', 'I&1', 'I&2', 'I&3', 'I&F', 'I&P', 'I&S', 'IOD'],
                                             [0,1,2,3,4,5],
                                             ['N', 'Y'],
                                             [0,1,2,3,4,5,6,7,8,9], 
                                             ['No Conviction', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'],
                                             [0,1,2,3],#Missing? Can't mix int and strings... what to do?
                                             ['Not in Employment', 'Armed Services', 'Bar/Restaurant', 'Building & Construction', 'Driving/Motor Trade', 'Education/Research', 'Farming/Animals', 'Government/Public Services', 'Healthcare', 'Leisure/Beauty', 'Manual/Unskilled', 'Office/Professional', 'Other', 'Security Cash & Courier', 'Skilled', 'Sports/Entertainment'],
                                             ['Common Law', 'Divorced', 'Married', 'Partnered', 'Separated', 'Single', 'N/A'],
                                             ['Full (UK)', 'International', 'Provisional', 'Foreign', 'Restricted < 3 Years', 'Restricted >= 3 Years'],
                                             ['N/A', 'N', 'Y'],
                                             ['Employed', 'Employed (Temporary)','Financially Assisted', 'Household Duties', 'In Full Or Part Time Education', 'Independent Means', 'Unemployed', 'Own Company', 'Retired', 'Self Employed', 'Voluntary Work', 'Contractor', 'Missing'],
                                             [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119],
                                             ['Agriculture & Animals', 'Antique / Arts Trade', 'Armed Services', 'Aviation', 'Blue Collar', 'Business Professional', 'Cash Collector', 'Child Care', 'Construction', 'Courier', 'Craftsman', 'Design', 'Education and Training ', 'Electrical', 'Emergency Services', 'Engineering', 'Entertainment & Media', 'Finance & Insurance', 'Food', 'Government and Agencies', 'Healthcare', 'Holiday', 'Hotel Pub & Restaurant', 'Installation', 'IT', 'Leisure & Beauty', 'Maintenance & Repair', 'Manufacturing', 'Marketing', 'Mining and Fuel', 'Not in Employment', 'Office Administration', 'Other', 'Parks & Gardens', 'Photography', 'Property', 'Public Service', 'Publishing', 'Retail Food', 'Retail Sales', 'Science', 'Security', 'Social Services', 'Telecommunications', 'Tradesman', 'Transport', 'Missing'],
                                             [0,1,2,3,4,5,6,7,8,9],
                                             ['0 to 1', '1 to 2', '2 to 3', '3 to 4', '4 to 5', 'No Conviction'],
                                             ['1 to 2', '2 to 3', '3 to 4', '4 to 5', '0 to 1', 'No Claim'],
                                             ['1 to 2', '2 to 3', '3 to 4', '4 to 5', '0 to 1', 'No Claim'],
                                             ['0 to 1', '1 to 2', '2 to 3', '3 to 4', '4 to 5', 'No Claim'],
                                             [17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], 
                                             ['AJ-NB', 'AJ_RC', 'NB', 'RB', 'RC'],
                                             ['No Claim', '1', '2', '3', '4', '5'],
                                             ['No Claim', '1', '2', '3', '4', '5'],
                                             ['No Claim', '1', '2', '3', '4', '5'],
                                             ['No Claim', '1', '2', '3', '4', '5'],
                                             ['No Claim', '1', '2', '3', '4', '5'],
                                             ['No Claim', '1', '2', '3', '4', '5+'], 
                                             [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
                                             ['Proposer', 'Spouse', 'Common-Law Partner', 'Family', 'Company', 'Leased Vehicle', 'Other'],
                                             ['Proposer', 'Spouse', 'Common-Law Partner', 'Family', 'Company', 'Leased Vehicle', 'Other'],
                                             ['Under 1000', '1000 - 1999', '2000 - 2999', '3000 - 3999', '4000 - 4999', '5000 - 5999', '6000 - 6999', '7000 - 7999', '8000 - 8999', '9000 - 9999', '10000 - 10999', '11000 - 11999', '12000 - 12999', '13000 - 13999', '14000 - 14999', '15000 - 15999', '16000 - 16999', '17000 - 17999', '18000 - 18999', '19000 - 19999', '20000-20999', '21000-21999', '22000-22999', '23000-23999', '24000-24999', '25000-25999', '26000-26999', '27000-27999', '28000-28999', '29000-29999', '30000-30999', '31000-31999', '32000-32999', '33000-33999', '34000-34999', '35000-35999', '36000-36999', '37000-37999', '38000-38999', '39000-39999', '40000-40999', '41000-41999', '42000-42999', '43000-43999', '44000-44999', '45000-45999', '46000-46999', '47000-47999', '48000-48999', '49000-49999', '50000+'],
                                             ['0', '1', '2', '3', '4', '5', '6+', 'Default'],
                                             ['0 to 99', '100 to 199', '200+', 'Default'],
                                             ['0-74', '75-84','85-94', '95-104', '105-114', '115-124', '125-134', '135-144', '145+', 'Default'],
                                             ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', 'Default']])
vehicle_value_encoded = ordinal_encoder.fit_transform(Saga_NB_dropped_unnecessary_df[[
                            'AgePassedTest', 'Cover', 'Duration', 'Garaged', 'NCDAllowed',
                            'NCDDifference', 'RegistrationYear', 'VehicleAgeatPurchase', 
                            'YearsOwned', 'Class_of_Use', 'Annual_Mileage', 'Age_Difference',
                            'Add_Driving_Experience', 'Add_Other_Vehs_Owned', 'Add_Driver_Age',
                            'Add_Access_to_Other_Vehs', 'Access_to_Other_Vehs', 'PDR_Code',
                            'Other_Vehs_Owned', 'NCD_Protected', 'NCD_Earned', 'Most_Severe_Conviction',
                            'Main_UK_Residency', 'Main_Occupation', 'Main_Marital_Status', 
                            'Main_Licence_Type', 'Main_Full_Time_Employed', 'Main_Employment_Type',
                            'Main_Driving_Experience_Months', 'Main_Employer_Business', 'Main_Driving_Experience',
                            'Latest_Conviction', 'Latest_Fault_Accident_Claim','Latest_Non_Fault_Accident_Claim',
                            'Latest_Windscreen_Claim','Main_Driver_Age', 'Transaction_Type', 'Tot_Windscreen_Claims', 
                            'Tot_Vandalism_Claims', 'Tot_Theft_Claims', 'Tot_Non_Fault_Accident_Claims', 
                            'Tot_Fire_Claims', 'Tot_Fault_Accident_Claims', 'Vehicle_Age', 'Vehicle_Keeper', 
                            'Vehicle_Owner', 'Vehicle_Value', 'Saga_Years_with_Broker', 'SagaCais', 'SagaFactor', 'SagaQuoteLag']])

In [ ]:
vehicle_value_encoded

In [ ]:
Saga_NB_transformed_df[[
                        'AgePassedTest', 'Cover', 'Duration', 'Garaged', 'NCDAllowed',
                        'NCDDifference', 'RegistrationYear', 'VehicleAgeatPurchase', 
                        'YearsOwned', 'Class_of_Use', 'Annual_Mileage', 'Age_Difference',
                        'Add_Driving_Experience', 'Add_Other_Vehs_Owned', 'Add_Driver_Age',
                        'Add_Access_to_Other_Vehs', 'Access_to_Other_Vehs', 'PDR_Code',
                        'Other_Vehs_Owned', 'NCD_Protected', 'NCD_Earned', 'Most_Severe_Conviction',
                        'Main_UK_Residency', 'Main_Occupation', 'Main_Marital_Status', 
                        'Main_Licence_Type', 'Main_Full_Time_Employed', 'Main_Employment_Type',
                        'Main_Driving_Experience_Months', 'Main_Employer_Business', 'Main_Driving_Experience',
                        'Latest_Conviction', 'Latest_Fault_Accident_Claim','Latest_Non_Fault_Accident_Claim',
                        'Latest_Windscreen_Claim','Main_Driver_Age', 'Transaction_Type', 'Tot_Windscreen_Claims', 
                        'Tot_Vandalism_Claims', 'Tot_Theft_Claims', 'Tot_Non_Fault_Accident_Claims', 
                        'Tot_Fire_Claims', 'Tot_Fault_Accident_Claims', 'Vehicle_Age', 'Vehicle_Keeper', 
                        'Vehicle_Owner', 'Vehicle_Value', 'Saga_Years_with_Broker', 'SagaCais', 'SagaFactor', 'SagaQuoteLag']] = vehicle_value_encoded
Saga_NB_dropped_unnecessary_df[[
                                'AgePassedTest', 'Cover', 'Duration', 'Garaged', 'NCDAllowed',
                                'NCDDifference', 'RegistrationYear', 'VehicleAgeatPurchase', 
                                'YearsOwned', 'Class_of_Use', 'Annual_Mileage', 'Age_Difference',
                                'Add_Driving_Experience', 'Add_Other_Vehs_Owned', 'Add_Driver_Age',
                                'Add_Access_to_Other_Vehs', 'Access_to_Other_Vehs', 'PDR_Code',
                                'Other_Vehs_Owned', 'NCD_Protected', 'NCD_Earned', 'Most_Severe_Conviction',
                                'Main_UK_Residency', 'Main_Occupation', 'Main_Marital_Status', 
                                'Main_Licence_Type', 'Main_Full_Time_Employed', 'Main_Employment_Type',
                                'Main_Driving_Experience_Months', 'Main_Employer_Business', 'Main_Driving_Experience',
                                'Latest_Conviction', 'Latest_Fault_Accident_Claim','Latest_Non_Fault_Accident_Claim',
                                'Latest_Windscreen_Claim','Main_Driver_Age', 'Transaction_Type', 'Tot_Windscreen_Claims', 
                                'Tot_Vandalism_Claims', 'Tot_Theft_Claims', 'Tot_Non_Fault_Accident_Claims', 
                                'Tot_Fire_Claims', 'Tot_Fault_Accident_Claims', 'Vehicle_Age', 'Vehicle_Keeper', 
                                'Vehicle_Owner', 'Vehicle_Value', 'Saga_Years_with_Broker', 'SagaCais', 'SagaFactor', 'SagaQuoteLag']] = Saga_NB_transformed_df[[
                                                                                                                                                                'AgePassedTest', 'Cover', 'Duration', 'Garaged', 'NCDAllowed',
                                                                                                                                                                'NCDDifference', 'RegistrationYear', 'VehicleAgeatPurchase', 'YearsOwned', 'Class_of_Use', 'Annual_Mileage', 'Age_Difference',
                                                                                                                                                                'Add_Driving_Experience', 'Add_Other_Vehs_Owned', 'Add_Driver_Age',
                                                                                                                                                                'Add_Access_to_Other_Vehs', 'Access_to_Other_Vehs', 'PDR_Code',
                                                                                                                                                                'Other_Vehs_Owned', 'NCD_Protected', 'NCD_Earned', 'Most_Severe_Conviction',
                                                                                                                                                                'Main_UK_Residency', 'Main_Occupation', 'Main_Marital_Status', 
                                                                                                                                                                'Main_Licence_Type', 'Main_Full_Time_Employed', 'Main_Employment_Type',
                                                                                                                                                                'Main_Driving_Experience_Months', 'Main_Employer_Business', 'Main_Driving_Experience',
                                                                                                                                                                'Latest_Conviction', 'Latest_Fault_Accident_Claim','Latest_Non_Fault_Accident_Claim',
                                                                                                                                                                'Latest_Windscreen_Claim','Main_Driver_Age', 'Transaction_Type', 'Tot_Windscreen_Claims', 
                                                                                                                                                                'Tot_Vandalism_Claims', 'Tot_Theft_Claims', 'Tot_Non_Fault_Accident_Claims', 
                                                                                                                                                                'Tot_Fire_Claims', 'Tot_Fault_Accident_Claims', 'Vehicle_Age', 'Vehicle_Keeper', 
                                                                                                                                                                'Vehicle_Owner', 'Vehicle_Value', 'Saga_Years_with_Broker', 'SagaCais', 'SagaFactor', 'SagaQuoteLag']]

In [ ]:
Saga_NB_dropped_unnecessary_df
column_headers = Saga_NB_dropped_unnecessary_df.columns

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# perform a robust scaler transform of the dataset
trans = MinMaxScaler()
Saga_NB_dropped_unneccessary_df_scaled = trans.fit_transform(Saga_NB_dropped_unnecessary_df)
Saga_NB_dropped_unneccessary_df = pd.DataFrame(Saga_NB_dropped_unneccessary_df_scaled)
Saga_NB_dropped_unnecessary_df.columns = column_headers
Saga_NB_dropped_unneccessary_df.head()

In [ ]:
x_train_stand = Saga_NB_dropped_unnecessary_df.drop(['cancelled'], axis=1)
y_train_stand = Saga_NB_dropped_unnecessary_df['cancelled'].copy()

### SelectKBest Feature Importance 

In [ ]:
from sklearn.feature_selection import SelectKBest 
from sklearn.feature_selection import f_regression 

kbest = SelectKBest(score_func=f_regression, k=15)
kbest.fit(x_train_stand, y_train_stand)

print("Feature scores", kbest.scores_)
print()
print("Selected Features", list(x_train_stand.columns[kbest.get_support()]))
print()
print("Removed Features", list(x_train_stand.columns[~kbest.get_support()]))


In [ ]:
C(AgePassedTest) + C(Main_Driver_Age) + C(Annual_Mileage) + C(Vehicle_Value) + C(VehicleAgeatPurchase) + C(YearsOwned) + C(SagaQuoteLag) + C(SagaFactor) + C(SagaCais) + C(Saga_Years_with_Broker) + C(Garaged) +  C(Add_Driver_Age) + C(Main_Occupation) + C(PDR_Code) + C(Class_of_Use) + C(Main_Marital_Status) + C(Latest_Fault_Accident_Claim) + C(NCD_Protected) + C(Main_Employment_Type) + C(NCDAllowed) + C(Transaction_Type) + C(Latest_Non_Fault_Accident_Claim)

### GLM Model Based on Features returned from SelectKBest Feature Importance 

In [ ]:
from sklearn.model_selection import train_test_split
X = Saga_NB_dropped_unnecessary_df.drop(columns=['cancelled'])


y = Saga_NB_dropped_unnecessary_df['cancelled']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 123)

feature_names = [val for val in X]


In [ ]:
factors_string_selectkbest = "'AgePassedTest', 'NCDAllowed', 'VehicleAgeatPurchase', 'PDR_Code', 'NCD_Protected', 'NCD_Earned', 'Main_Occupation', 'Main_Full_Time_Employed', 'Main_Employment_Type', 'Main_Driver_Age', 'Vehicle_Age', 'Vehicle_Value', 'Saga_Years_with_Broker', 'SagaCais', 'SagaQuoteLag'"
factors_string_selectkbest.replace(",", " +").replace("'","")

In [ ]:
mod_banded = smf.glm(formula = "cancelled ~ (C(AgePassedTest) + C(NCDAllowed) + C(VehicleAgeatPurchase) + C(PDR_Code) + C(NCD_Protected) + C(NCD_Earned) + C(Main_Occupation) + C(Main_Full_Time_Employed) + C(Main_Employment_Type) + C(Main_Driver_Age) + C(Vehicle_Age) + C(Vehicle_Value) + C(Saga_Years_with_Broker) + C(SagaCais) + C(SagaQuoteLag))", data =Saga_NB_dropped_unnecessary_df , family=sm.families.Binomial()).fit()
mod_banded.summary()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
X =  Saga_NB_dropped_unnecessary_df[['AgePassedTest', 'NCDAllowed', 'VehicleAgeatPurchase', 'PDR_Code', 'NCD_Protected', 'NCD_Earned', 'Main_Occupation', 'Main_Full_Time_Employed', 'Main_Employment_Type', 'Main_Driver_Age', 'Vehicle_Age', 'Vehicle_Value', 'Saga_Years_with_Broker', 'SagaCais', 'SagaQuoteLag']]
X['Intercept'] = 1

vif = pd.DataFrame()
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range (X.shape[1])]
vif.sort_values(by="VIF", ascending=False) 

In [ ]:
## Removing Main_Employment_Type
mod_banded_remove1 = smf.glm(formula = "cancelled ~ (C(AgePassedTest) + C(NCDAllowed) + C(VehicleAgeatPurchase) + C(PDR_Code) + C(NCD_Protected) + C(NCD_Earned) + C(Main_Occupation) + C(Main_Full_Time_Employed)  + C(Main_Driver_Age) + C(Vehicle_Age) + C(Vehicle_Value) + C(Saga_Years_with_Broker) + C(SagaCais) + C(SagaQuoteLag))", data =Saga_NB_dropped_unnecessary_df , family=sm.families.Binomial()).fit()
mod_banded_remove1.summary()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
X =  Saga_NB_dropped_unnecessary_df[['AgePassedTest', 'NCDAllowed', 'VehicleAgeatPurchase', 'PDR_Code', 'NCD_Protected', 'NCD_Earned', 'Main_Occupation', 'Main_Full_Time_Employed', 'Main_Driver_Age', 'Vehicle_Age', 'Vehicle_Value', 'Saga_Years_with_Broker', 'SagaCais', 'SagaQuoteLag']]
X['Intercept'] = 1

vif = pd.DataFrame()
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range (X.shape[1])]
vif.sort_values(by="VIF", ascending=False) 

In [ ]:
## Removing NCD_Earned
mod_banded_remove2 = smf.glm(formula = "cancelled ~ (C(AgePassedTest) + C(NCDAllowed) + C(VehicleAgeatPurchase) + C(PDR_Code) + C(NCD_Protected) + C(Main_Occupation) + C(Main_Full_Time_Employed)  + C(Main_Driver_Age) + C(Vehicle_Age) + C(Vehicle_Value) + C(Saga_Years_with_Broker) + C(SagaCais) + C(SagaQuoteLag))", data =Saga_NB_dropped_unnecessary_df , family=sm.families.Binomial()).fit()
#mod_banded_remove2.summary()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
X =  Saga_NB_dropped_unnecessary_df[['AgePassedTest', 'NCDAllowed', 'VehicleAgeatPurchase', 'PDR_Code', 'NCD_Protected', 'Main_Occupation', 'Main_Full_Time_Employed', 'Main_Driver_Age', 'Vehicle_Age', 'Vehicle_Value', 'Saga_Years_with_Broker', 'SagaCais', 'SagaQuoteLag']]
X['Intercept'] = 1

vif = pd.DataFrame()
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range (X.shape[1])]
vif.sort_values(by="VIF", ascending=False) 

### Gini Calculation 

In [ ]:
from sklearn import metrics
y_predict = mod_banded_remove2.predict(X_test)

roc_auc = metrics.roc_auc_score(y_test, y_predict)
Gini = 2*roc_auc - 1
print(Gini)

In [ ]:
import matplotlib.pyplot as plt
import scipy.interpolate
import scipy.integrate

def gini(actual, pred):
    assert (len(actual) == len(pred))
    all = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)
    all = all[np.lexsort((all[:, 2], -1 * all[:, 1]))]
    totalLosses = all[:, 0].sum()
    giniSum = all[:, 0].cumsum().sum() / totalLosses

    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)


def gini_normalized(actual, pred):
    return gini(actual, pred) / gini(actual, actual)

In [ ]:
gini_predictions = gini(y_test, y_predict)
gini_max = gini(y_test, y_test)
ngini= gini_normalized(y_test, y_predict)
print('Gini: %.3f, Max. Gini: %.3f, Normalized Gini: %.3f' % (gini_predictions, gini_max, ngini))